In [1]:
import numpy as np
from sklearn.datasets import load_iris

iris = load_iris()
X = iris.data[:, 2:4]   # petal length, petal width
y = iris.target

def entropy(y):
    values, counts = np.unique(y, return_counts=True)
    probs = counts / counts.sum()
    return -np.sum(probs * np.log2(probs))

def information_gain(y, y_left, y_right):
    H_parent = entropy(y)
    w_left = len(y_left) / len(y)
    w_right = len(y_right) / len(y)
    return H_parent - (w_left * entropy(y_left) + w_right * entropy(y_right))

def best_split(X, y):
    best_ig = -1
    best_feature = None
    best_threshold = None

    for feature in range(X.shape[1]):
        thresholds = np.unique(X[:, feature])
        for t in thresholds:
            left = y[X[:, feature] <= t]
            right = y[X[:, feature] > t]

            if len(left) == 0 or len(right) == 0:
                continue

            ig = information_gain(y, left, right)

            if ig > best_ig:
                best_ig = ig
                best_feature = feature
                best_threshold = t

    return best_feature, best_threshold, best_ig

def build_tree(X, y, depth=0, max_depth=2):
    if len(np.unique(y)) == 1 or depth == max_depth:
        return np.bincount(y).argmax()

    feature, threshold, ig = best_split(X, y)

    if feature is None:
        return np.bincount(y).argmax()

    print("  " * depth + f"Split: Feature {feature} <= {threshold:.2f}")

    left_idx = X[:, feature] <= threshold
    right_idx = X[:, feature] > threshold

    left_tree = build_tree(X[left_idx], y[left_idx], depth + 1, max_depth)
    right_tree = build_tree(X[right_idx], y[right_idx], depth + 1, max_depth)

    return (feature, threshold, left_tree, right_tree)

tree = build_tree(X, y)

print("\nFinal Tree Structure:")
print(tree)


Split: Feature 0 <= 1.90
  Split: Feature 1 <= 1.70

Final Tree Structure:
(0, 1.9, 0, (1, 1.7, 1, 2))
